In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import KNNImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price-bangladesh/house_price_bd.csv


In [2]:
data = pd.read_csv("/kaggle/input/house-price-bangladesh/house_price_bd.csv")
data.head(5)

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_taka,Location
0,We Are Offering You A Very Spacious 1960 Sq Ft...,3.0,4.0,3,vacant,1960.0,dhaka,"৳39,000,000","Gulshan 1, Gulshan"
1,Valuable 1705 Square Feet Apartment Is Ready T...,3.0,3.0,1,vacant,1705.0,dhaka,"৳16,900,000","Lake Circus Road, Kalabagan"
2,1370 square feet apartment is ready to sale in...,3.0,3.0,6,vacant,1370.0,dhaka,"৳12,500,000","Shukrabad, Dhanmondi"
3,2125 Square Feet Apartment For Sale In Bashund...,3.0,3.0,4,vacant,2125.0,dhaka,"৳20,000,000","Block L, Bashundhara R-A"
4,Buy This 2687 Square Feet Flat In The Nice Are...,3.0,3.0,4,vacant,2687.0,dhaka,"৳47,500,000","Road No 25, Banani"


In [3]:
data.shape

(3865, 9)

In [4]:
data.describe()

,Bedrooms,Bathrooms,Floor_area
count,2864.000000,2864.000000,3766.000000
mean,3.133031,2.992668,1940.299522
std,2.215457,0.978434,6024.921935
min,1.000000,1.000000,84.000000
25%,3.000000,3.000000,1100.000000
50%,3.000000,3.000000,1380.000000
75%,3.000000,3.000000,1860.000000
max,50.000000,10.000000,195840.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865 entries, 0 to 3864
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             3865 non-null   object 
 1   Bedrooms          2864 non-null   float64
 2   Bathrooms         2864 non-null   float64
 3   Floor_no          3181 non-null   object 
 4   Occupancy_status  3766 non-null   object 
 5   Floor_area        3766 non-null   float64
 6   City              3865 non-null   object 
 7   Price_in_taka     3865 non-null   object 
 8   Location          3859 non-null   object 
dtypes: float64(3), object(6)
memory usage: 271.9+ KB


In [6]:
data.dtypes

Title                object
Bedrooms            float64
Bathrooms           float64
Floor_no             object
Occupancy_status     object
Floor_area          float64
City                 object
Price_in_taka        object
Location             object
dtype: object

In [7]:
data.isna().sum()

Title                  0
Bedrooms            1001
Bathrooms           1001
Floor_no             684
Occupancy_status      99
Floor_area            99
City                   0
Price_in_taka          0
Location               6
dtype: int64

In [8]:
# check duplicates
data.duplicated().sum()

934

In [9]:
# Drop duplicate rows
data.drop_duplicates(inplace=True)

In [10]:
# check again
data.duplicated().sum()

0

In [11]:
# Columns of interest
columns_of_interest = ['Bedrooms', 'Bathrooms', 'Floor_no', 'Occupancy_status']

# Extract and analyze specific columns
for col in columns_of_interest:
    if col in data.columns:
        unique_values = data[col].unique()
        print(f"Column '{col}' unique values: {unique_values}")

Column 'Bedrooms' unique values: [ 3.  4.  2. nan 16.  8.  5.  1.  6. 18. 46.  7. 13. 33. 50. 23. 21. 22.
 20. 27. 15.]
Column 'Bathrooms' unique values: [ 4.  3.  2. nan  5. 10.  1.  6.  8.]
Column 'Floor_no' unique values: ['3' '1' '6' '4' '8' '2' '5' '7' '9' '8th' nan '10' '14' 'G+7' '13' '12'
 '1st' '11' 'A1,A2,A3,A4,A5,A6,A7' '0+7' '1F' '4th to 8th Backside' '18'
 '5th' '17' 'Merin City - Purbach']
Column 'Occupancy_status' unique values: ['vacant' 'occupied' nan]


In [12]:
data['Price_in_taka'] = data['Price_in_taka'].str.replace(',', '').str.replace('৳', '').astype("int64")

In [13]:
data['Price_in_taka'] = data['Price_in_taka'] / 1_000_000

In [14]:
data.rename(columns={'Price_in_taka': 'Price_in_millions'}, inplace=True)

In [15]:
data.head()

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_millions,Location
0,We Are Offering You A Very Spacious 1960 Sq Ft...,3.0,4.0,3,vacant,1960.0,dhaka,39.0,"Gulshan 1, Gulshan"
1,Valuable 1705 Square Feet Apartment Is Ready T...,3.0,3.0,1,vacant,1705.0,dhaka,16.9,"Lake Circus Road, Kalabagan"
2,1370 square feet apartment is ready to sale in...,3.0,3.0,6,vacant,1370.0,dhaka,12.5,"Shukrabad, Dhanmondi"
3,2125 Square Feet Apartment For Sale In Bashund...,3.0,3.0,4,vacant,2125.0,dhaka,20.0,"Block L, Bashundhara R-A"
4,Buy This 2687 Square Feet Flat In The Nice Are...,3.0,3.0,4,vacant,2687.0,dhaka,47.5,"Road No 25, Banani"


In [16]:
data['Floor_no'].unique()

array(['3', '1', '6', '4', '8', '2', '5', '7', '9', '8th', nan, '10',
       '14', 'G+7', '13', '12', '1st', '11', 'A1,A2,A3,A4,A5,A6,A7',
       '0+7', '1F', '4th to 8th Backside', '18', '5th', '17',
       'Merin City - Purbach'], dtype=object)

In [17]:
data[data['Floor_no'] == 'Merin City - Purbach']

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_millions,Location
3313,Plot For Sale In A Suitable Place In Bproperty...,NaN,NaN,Merin City - Purbach,vacant,2160.0,narayanganj-city,4.575,"Rupganj, Narayanganj"
3388,"For Sale, Residential Plot Is Situated In Bpro...",NaN,NaN,Merin City - Purbach,vacant,2160.0,narayanganj-city,4.575,"Rupganj, Narayanganj"
3392,Plot For Sale In A Suitable Place In Bproperty...,NaN,NaN,Merin City - Purbach,vacant,2160.0,narayanganj-city,4.575,"Rupganj, Narayanganj"
3393,Plot For Sale In A Suitable Place In Bproperty...,NaN,NaN,Merin City - Purbach,vacant,2160.0,narayanganj-city,4.575,"Rupganj, Narayanganj"
3432,In The Beautiful Location Of Bproperty Village...,NaN,NaN,Merin City - Purbach,vacant,2160.0,narayanganj-city,4.575,"Rupganj, Narayanganj"
3492,Plot For Sale In A Suitable Place In Narayanga...,NaN,NaN,Merin City - Purbach,vacant,2160.0,narayanganj-city,4.575,"Rupganj, Narayanganj"


In [18]:
data[data['Floor_no'] == 'A1,A2,A3,A4,A5,A6,A7']

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_millions,Location
1663,A Full Building Is For Sale In Sugandha Reside...,18.0,10.0,"A1,A2,A3,A4,A5,A6,A7",vacant,10890.0,chattogram,105.0,"Sugandha Residential Area, Panchlaish"


In [19]:
data[data['Floor_no'] == '4th to 8th Backside']

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_millions,Location
2674,An Apartment Is Up For Sale In Dakshin Kattali...,3.0,3.0,4th to 8th Backside,vacant,1250.0,chattogram,3.8,"Dakshin Kattali, 11 No. South Kattali Ward"


In [20]:
print(data.shape)

(2931, 9)


In [21]:
# Step 1: Fix `Floor_no` values
def clean_floor_no(value):
    if pd.isna(value):
        return value
    value = str(value).strip().lower()
    if value.endswith('th') or value.endswith('st') or value.endswith('f'):
        return int(''.join(filter(str.isdigit, value)))
    if value == 'g+7' or value == '0+7':
        return 7
    if value == 'a1,a2,a3,a4,a5,a6,a7':
        return 'A1-A7'  # Placeholder for later row expansion
    if value == '4th to 8th backside':
        return '4-8'  # Placeholder for later row expansion
    if value == 'Merin City - Purbach':
        return np.nan
    return value

data['Floor_no'] = data['Floor_no'].apply(clean_floor_no)


In [22]:
# Step 2: Expand rows for specific cases
# Case: 'A1,A2,A3,A4,A5,A6,A7'
a1_a7_data = data[data['Floor_no'] == 'A1-A7'].copy()
a1_a7_expanded = pd.DataFrame()

for i in range(1, 8):
    temp = a1_a7_data.copy()
    temp['Floor_no'] = i
    a1_a7_expanded = pd.concat([a1_a7_expanded, temp], ignore_index=True)

In [23]:
# Case: '4th to 8th Backside'
four_to_eight_data = data[data['Floor_no'] == '4-8'].copy()
four_to_eight_expanded = pd.DataFrame()

for i in range(4, 9):
    temp = four_to_eight_data.copy()
    temp['Floor_no'] = i
    four_to_eight_expanded = pd.concat([four_to_eight_expanded, temp], ignore_index=True)

In [24]:
# Remove rows with placeholders and add expanded rows
data = data[~data['Floor_no'].isin(['A1-A7', '4-8'])]
data = pd.concat([data, a1_a7_expanded, four_to_eight_expanded], ignore_index=True)

# Convert `Floor_no` to numeric
data['Floor_no'] = pd.to_numeric(data['Floor_no'], errors='coerce')

In [25]:
data.head()

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_millions,Location
0,We Are Offering You A Very Spacious 1960 Sq Ft...,3.0,4.0,3.0,vacant,1960.0,dhaka,39.0,"Gulshan 1, Gulshan"
1,Valuable 1705 Square Feet Apartment Is Ready T...,3.0,3.0,1.0,vacant,1705.0,dhaka,16.9,"Lake Circus Road, Kalabagan"
2,1370 square feet apartment is ready to sale in...,3.0,3.0,6.0,vacant,1370.0,dhaka,12.5,"Shukrabad, Dhanmondi"
3,2125 Square Feet Apartment For Sale In Bashund...,3.0,3.0,4.0,vacant,2125.0,dhaka,20.0,"Block L, Bashundhara R-A"
4,Buy This 2687 Square Feet Flat In The Nice Are...,3.0,3.0,4.0,vacant,2687.0,dhaka,47.5,"Road No 25, Banani"


In [26]:
data.shape

(2941, 9)

In [27]:
data[data['Location']=='Dakshin Kattali, 11 No. South Kattali Ward']

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_millions,Location
1243,"Start Your New Home, In This Reasonable And Co...",4.0,3.0,5.0,vacant,1500.0,chattogram,6.0,"Dakshin Kattali, 11 No. South Kattali Ward"
1258,Visit This 825 Sq Ft Flat For Sale In Dakshin ...,3.0,3.0,3.0,vacant,825.0,chattogram,3.7,"Dakshin Kattali, 11 No. South Kattali Ward"
1981,1200 Sq Ft Flat With Great Residential Service...,3.0,3.0,4.0,vacant,1200.0,chattogram,8.0,"Dakshin Kattali, 11 No. South Kattali Ward"
2936,An Apartment Is Up For Sale In Dakshin Kattali...,3.0,3.0,4.0,vacant,1250.0,chattogram,3.8,"Dakshin Kattali, 11 No. South Kattali Ward"
2937,An Apartment Is Up For Sale In Dakshin Kattali...,3.0,3.0,5.0,vacant,1250.0,chattogram,3.8,"Dakshin Kattali, 11 No. South Kattali Ward"
2938,An Apartment Is Up For Sale In Dakshin Kattali...,3.0,3.0,6.0,vacant,1250.0,chattogram,3.8,"Dakshin Kattali, 11 No. South Kattali Ward"
2939,An Apartment Is Up For Sale In Dakshin Kattali...,3.0,3.0,7.0,vacant,1250.0,chattogram,3.8,"Dakshin Kattali, 11 No. South Kattali Ward"
2940,An Apartment Is Up For Sale In Dakshin Kattali...,3.0,3.0,8.0,vacant,1250.0,chattogram,3.8,"Dakshin Kattali, 11 No. South Kattali Ward"


In [28]:
data['Floor_no'].unique()

array([ 3.,  1.,  6.,  4.,  8.,  2.,  5.,  7.,  9., nan, 10., 14., 13.,
       12., 11., 18., 17.])

In [29]:
data.isna().sum()

Title                  0
Bedrooms             831
Bathrooms            831
Floor_no             581
Occupancy_status      89
Floor_area            89
City                   0
Price_in_millions      0
Location               6
dtype: int64

In [30]:
# unique values of "City"
data["City"].unique()

array(['dhaka', 'chattogram', 'cumilla', 'narayanganj-city', 'gazipur'],
      dtype=object)

In [31]:
columns_to_drop_miising_values = ['Occupancy_status', 'Floor_area', 'Location', 'Floor_no']

# Remove rows with missing values in the specified columns
data = data.dropna(subset=columns_to_drop_miising_values, axis = 0)
# verify the result
print(data.isnull().sum())

Title                  0
Bedrooms             270
Bathrooms            270
Floor_no               0
Occupancy_status       0
Floor_area             0
City                   0
Price_in_millions      0
Location               0
dtype: int64


In [32]:
# Impute the missing values in Bathrooms and Bedrooms using Floor area
features = ['Floor_area', 'Bedrooms', 'Bathrooms']

imputer = KNNImputer(n_neighbors=5, weights='distance')
data[features] = imputer.fit_transform(data[features])

In [33]:
data.isna().sum()

Title                0
Bedrooms             0
Bathrooms            0
Floor_no             0
Occupancy_status     0
Floor_area           0
City                 0
Price_in_millions    0
Location             0
dtype: int64

In [34]:
for col in data.columns:
    if pd.api.types.is_numeric_dtype(data[col]):
        print(f"Top 5 largest values in '{col}':")
        print(data[col].nlargest(5))

Top 5 largest values in 'Bedrooms':
217     46.000000
99      27.803983
2250    27.000000
1625    22.000000
1457    21.000000
Name: Bedrooms, dtype: float64
Top 5 largest values in 'Bathrooms':
2047    10.0
206     10.0
217     10.0
986     10.0
999     10.0
Name: Bathrooms, dtype: float64
Top 5 largest values in 'Floor_no':
2110    18.0
2161    17.0
198     14.0
1560    14.0
230     13.0
Name: Floor_no, dtype: float64
Top 5 largest values in 'Floor_area':
1489    24000.0
99      21500.0
217     19200.0
40      16000.0
510     15120.0
Name: Floor_area, dtype: float64
Top 5 largest values in 'Price_in_millions':
1602    363.2
1910    363.2
2023    363.2
1245    350.0
2128    350.0
Name: Price_in_millions, dtype: float64


In [35]:
for col in data.columns:
    print(data[col].value_counts().head(5))

Title
A Full Building Is For Sale In Sugandha Residential Area, Panchlaish.           7
An Apartment Is Up For Sale In Dakshin Kattali, Near Anandomoyee Kali Temple    5
Buy This 240 Sq Ft Shop At Bakalia                                              2
1445 Sq Ft Residential Apartment Is Available For Sale In Chotora, Cumilla      2
Visit This Apartment For Sale In Bakalia Near Haji Eqbal Jame Masjid            1
Name: count, dtype: int64
Bedrooms
3.000000    1622
2.000000     294
4.000000     167
1.316456      16
1.227586      15
Name: count, dtype: int64
Bathrooms
3.0     1320
2.0      465
4.0      235
5.0       55
10.0      18
Name: count, dtype: int64
Floor_no
1.0    382
4.0    308
5.0    296
2.0    275
3.0    262
Name: count, dtype: int64
Occupancy_status
vacant      2357
occupied       2
Name: count, dtype: int64
Floor_area
1300.0    69
1500.0    68
1250.0    62
1200.0    59
1400.0    56
Name: count, dtype: int64
City
chattogram          1077
dhaka               1006
gazipur      

In [36]:
averages = data.groupby('Location').mean(numeric_only=True)
averages.sort_values(by=averages.columns[4], ascending=False).head(10)

,Bedrooms,Bathrooms,Floor_no,Floor_area,Price_in_millions
Location,,,,,
"Motijheel Road, Motijheel",4.201357,4.583595,1.000000,4686.000000,210.8700
"28 No. Pathantooly Ward, Double Mooring",9.062454,5.765657,4.461538,6678.615385,147.3500
"Kazi Nazrul Islam Avenue, Banglamotors",4.000000,5.000000,1.000000,3000.000000,120.0000
"Hemayetpur, Savar",15.401991,5.875040,6.500000,11200.000000,111.4500
"New Eskaton Road, Eskaton",3.914478,4.768314,2.500000,2922.500000,100.7500
"Gulshan 2, Gulshan",4.000000,4.000000,1.500000,3655.000000,93.9000
"CDA Avenue, East Nasirabad",7.500104,5.500104,5.000000,4967.500000,88.3750
"Sheikh Mujib Road, Double Mooring",7.946862,6.359795,9.750000,6158.500000,81.1855
"Sugandha Residential Area, Panchlaish",12.545455,7.636364,3.909091,7619.818182,72.4680
